In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

PrOJECT_ROOT = Path.cwd()
RAW_DATA_DIR = PrOJECT_ROOT / "data" / "raw"
PROCESSED_DATA_DIR = PrOJECT_ROOT / "data" / "processed"

PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

products = pd.read_csv(RAW_DATA_DIR / "products.csv")
prior = pd.read_csv(RAW_DATA_DIR / "order_products__prior.csv")

In [2]:
products.info()

<class 'pandas.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   product_id     49688 non-null  int64
 1   product_name   49688 non-null  str  
 2   aisle_id       49688 non-null  int64
 3   department_id  49688 non-null  int64
dtypes: int64(3), str(1)
memory usage: 3.0 MB


In [3]:
prior.info()

<class 'pandas.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB


In [4]:
df = prior.merge(products, on="product_id", how="inner")
df.sample(5)

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
32387109,3416166,46516,4,0,Sesame Bagels,93,3
25727059,2713284,33768,4,1,Sinfully Sweet Campari Tomatoes,83,4
14262865,1505164,34864,1,1,Pet Waste Bags,40,8
31773614,3351195,20015,20,1,Grandma's Hummus,67,20
22136169,2334804,48423,6,0,Organic Sweet Peas,116,1


In [5]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 7 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
 4   product_name       str  
 5   aisle_id           int64
 6   department_id      int64
dtypes: int64(6), str(1)
memory usage: 2.5 GB


In [ ]:
# We only need the order_id and product_id columns for the FP-Growth algorithm, so we will drop the other columns.
df_fp = df[["order_id", "product_id"]]
df_fp.sample(5)

,order_id,product_id
16389318,1728955,42964
29006545,3059228,17862
10381825,1096061,29487
12781439,1349062,17858
3486100,368325,45073


In [7]:
df_fp.info()

<class 'pandas.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 2 columns):
 #   Column      Dtype
---  ------      -----
 0   order_id    int64
 1   product_id  int64
dtypes: int64(2)
memory usage: 494.9 MB


In [ ]:
# Check for missing values
df_fp.isnull().sum()

order_id      0
product_id    0
dtype: int64

In [ ]:
# Check for duplicated rows
print("Number of duplicated rows:", df_fp.duplicated().sum())

Number of duplicated rows: 0


In [ ]:
# Check the number of unique orders and products
print(f"Number of unique orders: {df_fp['order_id'].nunique()}")
print(f"Number of unique products: {df_fp['product_id'].nunique()}")

Number of unique orders: 3214874
Number of unique products: 49677


In [ ]:
# Save the cleaned DataFrame to a parquet file for later use in the FP-Growth algorithm.
df_fp.to_parquet(
    PROCESSED_DATA_DIR / "transactions.parquet",
    engine="pyarrow",
    compression="snappy",
    index=False
)
# pyarrow is a fast and efficient library for reading and writing parquet files, while snappy is a compression algorithm that provides a good balance between compression ratio and speed. Setting index=False ensures that the DataFrame index is not included in the output file, which can save space and improve performance when reading the file later.

In [ ]:
# loading the data which is saved in parquet format
df_fp = pd.read_parquet(PROCESSED_DATA_DIR / "transactions.parquet")

In [13]:
df_fp.info()

<class 'pandas.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 2 columns):
 #   Column      Dtype
---  ------      -----
 0   order_id    int64
 1   product_id  int64
dtypes: int64(2)
memory usage: 494.9 MB


In [14]:
# Calculate basket sizes
basket_sizes = df_fp.groupby('order_id').size()

# Filter for orders with 2 or more products
multi_item_ids = basket_sizes[basket_sizes >= 2].index
df_mining = df_fp[df_fp['order_id'].isin(multi_item_ids)]

# Print metrics
print(f"Baskets with 1 item (Removed): {len(basket_sizes) - len(multi_item_ids):,}")
print(f"Baskets with 2+ items (Retained): {len(multi_item_ids):,}")

Baskets with 1 item (Removed): 156,748
Baskets with 2+ items (Retained): 3,058,126


## Data Quality Filtering: Handling "Single-Item" Baskets

### **Objective**
In **Association Rule Mining** (FP-Growth/Apriori), we seek to discover relationships between products (e.g., "If a customer buys A, they are likely to buy B"). For a relationship to exist, an order must contain at least **two items**. 

### **The Logic**
Orders containing only one item—while useful for calculating general product popularity—provide zero information regarding **product affinity**. Including them in the mining phase:
1.  **Dilutes Support Metrics:** It makes items look less "associative" than they actually are.
2.  **Increases Computational Overhead:** The algorithm spends time processing millions of rows that cannot physically produce a rule.

### **Implementation**
We group the data by `order_id` to calculate the "basket size" for every transaction. We then create a mask to retain only the orders that meet our **minimum complexity requirement** ($N \ge 2$).



### **output**
* **Baskets Removed:** 156,748 (Single-item "noise")
* **Baskets Retained:** 3,058,126 (High-quality multi-item transactions)
* **Data Integrity:** By removing these, we ensure our **Confidence** and **Lift** scores are calculated against a baseline of actual shopping "trips" where choices were made between multiple products.

In [15]:
# Create baskets for FP-Growth

# We group by order_id and turn the product_ids into a list
# This creates the exact format the FP-Growth library expects
baskets = df_mining.groupby('order_id')['product_id'].apply(list).tolist()

print(f"Created {len(baskets):,} baskets.")
print(f"Example of a single basket: {baskets[0]}")

Created 3,058,126 baskets.
Example of a single basket: [33120, 28985, 9327, 45918, 30035, 17794, 40141, 1819, 43668]


## Data Structuring: Generating Transactional Baskets

### **Objective**
To convert our relational DataFrame into a **Transactional "List of Lists"** format. This is the final structural requirement before the data can be ingested by the **MLxtend** library and the **FP-Growth** algorithm.

### **The Transformation Logic**
We are shifting the data from a "Tall" format (where each row is one product) to a "Wide" format (where each row is one complete order).

1.  **`groupby('order_id')`**: This identifies all rows belonging to a single shopping trip.
2.  **`['product_id'].apply(list)`**: Within each group, we harvest all the unique product IDs and pack them into a standard Python list.
3.  **`.tolist()`**: We convert the resulting Pandas Series into a master list containing millions of sub-lists (one per order).

### **Why This Format?**
The **FP-Tree** (Frequent Pattern Tree) structure works by scanning through transactions one by one. By providing a list of lists, we allow the algorithm to:
* Identify overlapping items across different transactions efficiently.
* Build the tree nodes without the overhead of Pandas index lookups.
* Preserve the **Product IDs** as the primary values for mining.



### **Execution Metrics**
* **Total Baskets:** 3,058,126
* **Structure:** `[[ID1, ID2, ...], [ID3, ID4, ...], ...]`
* **Example Output:** A single basket now looks like a digital receipt: `[33120, 28985, 9327, 45918, ...]`.

In [16]:
# Sort the items within each basket
# This makes the FP-Tree construction much more efficient
baskets = [sorted(basket) for basket in baskets]

print(f"Example of a sorted basket: {baskets[0]}")

Example of a sorted basket: [1819, 9327, 17794, 28985, 30035, 33120, 40141, 43668, 45918]


In [18]:
# First, let's see what we're working with. 
# value_counts() gives us the frequency for every product in the dataset.
item_counts = df_mining['product_id'].value_counts()

# Let's see top 10 popular items
# If Bananas are at 15%, that's a huge anchor for our rules.
item_percents = (item_counts / len(baskets)) * 100
print(item_percents.head(10))

# Focus on the Top 2000.
# Why 2000? It covers the high-velocity items and keeps the matrix 
# small enough that my pc can handle it.
TOP_N = 2000
top_2000_set = set(item_counts.head(TOP_N).index)

print(f"\nLocked in the Top {TOP_N} products for the final matrix.")

product_id
24852    15.385828
13176    12.317216
21137     8.618448
21903     7.872632
47209     6.961355
47766     5.765067
47626     4.979062
16797     4.641176
26209     4.588693
27845     4.477154
Name: count, dtype: float64

Locked in the Top 2000 products for the final matrix.


## Feature Selection: The "Top-N" Frequency Filter

### **Objective**
To perform **Dimensionality Reduction** by isolating the most influential products in the dataset. This ensures our model focuses on high-velocity items that provide the strongest statistical "signal."

### **The "Long Tail" Logic**
In the Instacart dataset, product popularity follows a **Power Law distribution**. While there are ~50,000 unique products, a small percentage (the "Head") drives the majority of transactions, while thousands of products (the "Tail") appear in fewer than 100 orders.
* **Noise Reduction:** Rare items often create "spurious correlations"—coincidences that look like patterns but aren't statistically reliable.
* **Computational Optimization:** Reducing the feature space to 2,000 products allows us to build a **Sparse Matrix** that is small enough for local memory while still capturing the vast majority of consumer behavior.

### **Implementation Strategy**
1.  **Frequency Analysis**: We calculate the "Support" for every item by dividing its occurrence by the total number of baskets.
2.  **Top-K Selection**: we lock in the **Top 2,000 Product IDs** as our target features.
3.  **Set Optimization**: We convert these IDs into a Python `set`. This is a critical engineering step; checking if an item exists in a **Set** is significantly faster ($O(1)$) than checking a **List** ($O(N)$), which is vital when processing 32 million rows.

In [19]:
# Now I need to clean the actual baskets. 
# I'm looping through every order and tossing out anything that isn't a "Top 2000" item.
# Using a set lookup here because doing this 3 million times on a list would take time
# set lookups works like a hash table and are O(1) on average, while list lookups are O(n).
baskets_filtered = [[item for item in b if item in top_2000_set] for b in baskets]

# if I remove items, some baskets might now have only 1 item (or zero).
# I'll drop those now since you can't have an "association" with just one product.
baskets_final = [b for b in baskets_filtered if len(b) >= 2]

print(f"Cleaned up! We have {len(baskets_final):,} baskets left for the algorithm.")

Cleaned up! We have 2,761,063 baskets left for the algorithm.


## Data Pruning: Applying the Frequency Filter

### **Objective**
To synchronize our transactional baskets with our **Top-2000** feature selection. This step removes "rare" products from every order and performs a final quality check to ensure all remaining baskets are valid for mining.

1.  **High-Speed Filtering ($O(1)$ Lookup)**: 
    * We iterate through over 3 million baskets. Inside each basket, we check if a product exists in our `top_2000_set`. 
    * By using a **Python Set** (which functions like a Hash Table), the time complexity for this check is **Constant Time ($O(1)$)**. If we used a standard list, the processing time would be exponentially higher ($O(N)$).
2.  **Post-Pruning Validation**: 
    * Removing rare items can "hollow out" an order. For example, if a customer bought 1 popular banana and 2 rare spices, our filter removes the spices, leaving only the banana.
    * Since **FP-Growth** requires at least two items to form a relationship ($A \rightarrow B$), we perform a second pass to drop any baskets that were reduced to 0 or 1 items.



### **Why This Matters**
* **Dimensionality Alignment**: This ensures our final matrix has exactly 2,000 columns with no "orphaned" data.
* **Statistical Reliability**: By removing the "Long Tail" items from individual baskets, we ensure that the **Support** and **Confidence** metrics of our final rules are not skewed by infrequent, one-off purchases.

### **Current State**
* **Input**: 3,058,126 baskets (unfiltered).
* **Output**: 2,761,063 baskets (optimized).
* **Retention**: Approximately **90.3%** of multi-item baskets were retained after filtering for the Top 2,000 products, indicating that our feature selection covers the vast majority of core consumer behavior.

In [20]:
from mlxtend.preprocessing import TransactionEncoder

# I'm initializing the encoder to turn my list of lists into a True/False matrix.
te = TransactionEncoder()

print("Converting baskets to a sparse matrix.")

# The sparse=True argument is the most important part here. 
# It tells Python to only store the 'True' values, saving massive amounts of RAM.
# how sparse=True works: Instead of storing a full matrix with mostly False values.
te_ary = te.fit(baskets_final).transform(baskets_final, sparse=True)

# I'll wrap this in a Sparse DataFrame so I can keep the product_id column names.
df_encoded = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

print(f"Matrix is ready! Shape: {df_encoded.shape}")
print(f"Approximate RAM usage: {df_encoded.memory_usage().sum() / 1024**2:.2f} MB")

Converting baskets to a sparse matrix.
Matrix is ready! Shape: (2761063, 2000)
Approximate RAM usage: 100.64 MB


## Logical Transformation: Sparse Boolean Encoding

### **Objective**
To transform our list of baskets into a **Binary Matrix** (One-Hot Encoding). This is the specific mathematical format required by the **FP-Growth** algorithm to calculate item relationships.

### **The "Sparse" Solution**
In a standard matrix, the computer would create a cell for every product and every order. With 2.7 million orders and 2,000 products, that is **5.4 billion cells**. 
* **The Problem**: A "Dense" matrix would store every single `False` value, requiring over **20GB of RAM**.
* **The Solution**: We use `sparse=True`. This tells Python to only remember where the `True` values (the actual purchases) are located. 



### **Why This Step is Critical**
1. **Memory Efficiency**: By ignoring the empty space (the products a customer *didn't* buy), we reduced our RAM usage from ~20GB to approximately **100MB**.
2. **Preserving Identity**: We wrap the matrix in a **Sparse DataFrame**. This ensures that our column headers remain the original **Product IDs**, allowing us to identify the products later.
3. **Speed**: The `mlxtend` TransactionEncoder is highly optimized for this specific transformation, preparing the data for mining in seconds rather than minutes.

### **Technical Summary**
* **Input**: A list of lists (`baskets_final`).
* **Output**: A Compressed Sparse Row (CSR) matrix.
* **Result**: A highly compressed, high-speed data structure ready for the **FP-Growth** algorithm.

In [22]:
from mlxtend.frequent_patterns import fpgrowth

# The fpgrowth library has a bug with Sparse DataFrames when column names are integers.
# I'm converting the Product IDs to strings to bypass this limitation.
df_encoded.columns = [str(i) for i in df_encoded.columns]

# Now we try the mining again.
# I'm sticking with 1% (0.01) to see how many "strong" patterns we have.
print("Starting FP-Growth mining with string-based columns")
frequent_itemsets = fpgrowth(df_encoded, min_support=0.01, use_colnames=True)

# Sorting so the most popular itemsets are at the top.
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)

print(f"Success Found {len(frequent_itemsets)} frequent itemsets.")
print(frequent_itemsets.head(10))

Starting FP-Growth mining with string-based columns
Success Found 142 frequent itemsets.
     support            itemsets
14  0.168670  frozenset({24852})
6   0.135097  frozenset({13176})
15  0.094981  frozenset({21137})
2   0.086748  frozenset({21903})
7   0.076763  frozenset({47209})
16  0.063589  frozenset({47766})
39  0.054862  frozenset({47626})
51  0.050876  frozenset({16797})
66  0.050588  frozenset({26209})
8   0.049288  frozenset({27966})


## Pattern Discovery: Frequent Itemset Mining (FP-Growth)

### **Objective**
To identify **Frequent Itemsets**—groups of products that appear together in transactions more often than a specified threshold. We utilize the **FP-Growth (Frequent Pattern Growth)** algorithm for its superior speed and memory efficiency.

### **The Algorithm: Why FP-Growth?**
Unlike the older Apriori algorithm, FP-Growth does not generate "candidate" sets (potential combinations), which can be computationally expensive. Instead:
1. It builds a compressed **FP-Tree** structure in memory.
2. It fragments the tree into smaller "conditional" trees to extract frequent patterns.
This approach is significantly faster for large datasets like Instacart's 3-million-basket collection.



### **The Support Threshold**
We have set the `min_support` to **0.01 (1%)**. 
* **Meaning**: A product or combination of products must appear in at least **1% of all transactions** (approx. 27,610 baskets) to be included in our results.
* **Goal**: This high threshold ensures that we are looking at "core" consumer behaviors rather than rare, one-off occurrences.

### **Implementation Note: The Integer Bug Fix**
A technical workaround was required for this step. The `mlxtend` library has a known limitation when handling **Sparse DataFrames** with integer column names. 
* **The Fix**: We converted the **Product IDs** (e.g., `24852`) into **Strings** (e.g., `"24852"`). 
* **The Result**: This prevents the algorithm from confusing column labels with column indices, allowing the mining process to complete successfully.

### **Analysis of Results**
* **Success**: The algorithm identified **142 frequent itemsets**.
* **Top Itemsets**: As expected, the results are dominated by "anchor" products like Bananas and Organic Produce, providing a statistically sound foundation for the next phase: generating Association Rules.

In [23]:
from mlxtend.frequent_patterns import association_rules

# We generate the rules based on our frequent itemsets.
# I'm using 'lift' as the metric because it shows how much MORE likely 
# someone is to buy Y given X, compared to just buying Y randomly.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Let's sort them by 'lift' to find the strongest, most "surprising" relationships.
rules = rules.sort_values(by='lift', ascending=False)

print(f"Generated {len(rules)} association rules!")
# Showing the top 10 rules with the most important columns
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))

Generated 38 association rules!
           antecedents         consequents   support  confidence      lift
27  frozenset({27966})  frozenset({21137})  0.012264    0.248835  2.619848
26  frozenset({21137})  frozenset({27966})  0.012264    0.129126  2.619848
25  frozenset({24852})  frozenset({28204})  0.012293    0.072885  2.263507
24  frozenset({28204})  frozenset({24852})  0.012293    0.381785  2.263507
19  frozenset({13176})  frozenset({27966})  0.014669    0.108584  2.203071
18  frozenset({27966})  frozenset({13176})  0.014669    0.297628  2.203071
0   frozenset({47209})  frozenset({13176})  0.022579    0.294133  2.177204
1   frozenset({13176})  frozenset({47209})  0.022579    0.167129  2.177204
36  frozenset({45066})  frozenset({24852})  0.010289    0.358072  2.122917
37  frozenset({24852})  frozenset({45066})  0.010289    0.061000  2.122917


In [ ]:
# Load the product names
products = pd.read_csv(RAW_DATA_DIR /'products.csv')

# Create a dictionary for quick lookup
id_to_name = dict(zip(products['product_id'].astype(str), products['product_name']))

def get_names(ids_frozenset):
    # Converts the frozenset of ID strings back to a readable string of names
    return ", ".join([id_to_name.get(str(i), "Unknown") for i in ids_frozenset])

# Apply the naming function to our rules
rules['antecedents_names'] = rules['antecedents'].apply(get_names)
rules['consequents_names'] = rules['consequents'].apply(get_names)

# Display the Final Result
final_report = rules[['antecedents_names', 'consequents_names', 'support', 'confidence', 'lift']]
print("THE FINAL ASSOCIATION RULES")
print(final_report.head(10))

THE FINAL ASSOCIATION RULES
         antecedents_names       consequents_names   support  confidence  \
27     Organic Raspberries    Organic Strawberries  0.012264    0.248835   
26    Organic Strawberries     Organic Raspberries  0.012264    0.129126   
25                  Banana      Organic Fuji Apple  0.012293    0.072885   
24      Organic Fuji Apple                  Banana  0.012293    0.381785   
19  Bag of Organic Bananas     Organic Raspberries  0.014669    0.108584   
18     Organic Raspberries  Bag of Organic Bananas  0.014669    0.297628   
0     Organic Hass Avocado  Bag of Organic Bananas  0.022579    0.294133   
1   Bag of Organic Bananas    Organic Hass Avocado  0.022579    0.167129   
36        Honeycrisp Apple                  Banana  0.010289    0.358072   
37                  Banana        Honeycrisp Apple  0.010289    0.061000   

        lift  
27  2.619848  
26  2.619848  
25  2.263507  
24  2.263507  
19  2.203071  
18  2.203071  
0   2.177204  
1   2.177204  


## Final Results:

### **Objective**
To decode the mathematical output of the **FP-Growth** algorithm into a human-readable format. By mapping **Product IDs** back to their original names, we can identify actionable consumer trends and cross-selling opportunities.

### **The Decoding Logic**
We utilized the `products.csv` metadata to create a high-speed lookup dictionary. Since our association rules were generated using string-based IDs (to avoid the integer bug), we implemented a mapping function that:
1.  Accesses the `frozenset` of IDs for both **Antecedents** (the "If" item) and **Consequents** (the "Then" item).
2.  Retrieves the corresponding **Product Name** from our dictionary.
3.  Reformats the data into a clean, presentation-ready report.

### **Key Metrics Explained**
* **Support**: The percentage of all orders that contain both items. (e.g., ~1.2% for Organic Berries).
* **Confidence**: The probability that the consequent is purchased given that the antecedent is already in the cart. 
    * *Insight*: **38%** of customers who buy **Organic Fuji Apples** also buy a **Banana**.
* **Lift**: The strength of the association. 
    * *Insight*: A lift of **2.62** for **Organic Raspberries** and **Strawberries** means they are bought together **2.6 times more often** than if customers were picking items at random.

[Image of association rule mining metrics showing support confidence and lift relationships]

### **Final Analysis & Business Recommendations**
The results reveal a powerful **"Organic Cluster."** Customers purchasing organic produce are highly likely to stick within the organic ecosystem for their entire basket. 

**Strategic Recommendations:**
1.  **Digital Merchandising**: Place "Organic Raspberries" as a recommended item on the product page for "Organic Strawberries" to capitalize on the 2.6x Lift.
2.  **Bundle Promotions**: Create an "Organic Fruit Bowl" discount bundle featuring the top-performing pairs (Berries + Bananas).
3.  **Store Layout**: In physical or "Dark Store" fulfillment centers, keep **Organic Hass Avocados** and **Bag of Organic Bananas** in close proximity to reduce picking time and encourage multi-item purchases.